# #30DayMapChallenge
## Day 10: Air

### Data Source
1. ERA5 monthly averaged data on single levels from 1940 to present: https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means?tab=overview
- Product type: Monthly averaged reanalysis; Monthly averaged reanalysis by hour of day; Monthly averaged ensemble members; Monthly averaged ensemble members by hour of day
- Wind: 10m v-component of wind; 10m u-component of wind
- Year: 2010; 2011; 2012; 2013; 2014; 2015
- Time: 0000; 0100; 0200; 0300; 0400; 0500; 0600; 0700; 0800; 0900; 1000; 1100; 1200; 1300; 1400; 1500; 1600; 1700; 1800; 1900; 2000; 2100; 2200; 2300
- Whole available region

Product Type | Time Resolution	| Ensemble Info | Values per Month
- Monthly Averaged Reanalysis	| None (monthly mean) | No | 1
- Monthly Averaged Reanalysis by Hour of Day | Hourly (UTC) | No | 24
- Monthly Averaged Ensemble Members | None (monthly mean) | Yes (10 members) | 10
- Monthly Averaged Ensemble Members by Hour of Day | Every 3 hours (UTC) | Yes (10 members) | 80 (8×10)

### Setup

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [29]:
# Monthly Averaged Reanalysis
# Load file
ds1 = xr.open_dataset("data_stream-moda_stepType-avgua.nc")

# Print variable names
print(ds1.variables)

Frozen({'u10': <xarray.Variable (valid_time: 72, latitude: 721, longitude: 1440)> Size: 299MB
[74753280 values with dtype=float32]
Attributes: (12/32)
    GRIB_paramId:                             165
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      1038240
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgua
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_units:                               m s**-1
    long_name:                                10 metre U wind component
    units:                                    m s**-1
    standard_name:                            unknown
    GRIB_surface:                             0.0, 'v10': <xarray.Variable (valid_time: 72, latitude: 721, longitude: 1440)> Size: 299MB
[74753280 values with dtype=float32]
Attributes: (12/32)
    GRIB_paramId

In [37]:
# Monthly Averaged Reanalysis by Hour of Day
# Load one file
ds2 = xr.open_dataset("data_stream-mnth_stepType-avgua.nc")

# Print variable names
print(ds2.variables)

Frozen({'u10': <xarray.Variable (valid_time: 1728, latitude: 721, longitude: 1440)> Size: 7GB
[1794078720 values with dtype=float32]
Attributes: (12/32)
    GRIB_paramId:                             165
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      1038240
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgua
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_units:                               m s**-1
    long_name:                                10 metre U wind component
    units:                                    m s**-1
    standard_name:                            unknown
    GRIB_surface:                             0.0, 'v10': <xarray.Variable (valid_time: 1728, latitude: 721, longitude: 1440)> Size: 7GB
[1794078720 values with dtype=float32]
Attributes: (12/32)
    GRIB_par

In [25]:
# Monthly Averaged Ensemble Members
# Load file
ds3 = xr.open_dataset("data_stream-edmo_stepType-avgua.nc")

# Print variable names
print(ds3.variables)

Frozen({'u10': <xarray.Variable (number: 10, valid_time: 72, latitude: 361, longitude: 720)> Size: 749MB
[187142400 values with dtype=float32]
Attributes: (12/32)
    GRIB_paramId:                             165
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      259920
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgua
    ...                                       ...
    GRIB_totalNumber:                         10
    GRIB_units:                               m s**-1
    long_name:                                10 metre U wind component
    units:                                    m s**-1
    standard_name:                            unknown
    GRIB_surface:                             0.0, 'v10': <xarray.Variable (number: 10, valid_time: 72, latitude: 361, longitude: 720)> Size: 749MB
[187142400 values with dtype=float32]
Attributes: 

In [33]:
# Monthly Averaged Ensemble Members by Hour of Day
# Load one file
ds4 = xr.open_dataset("data_stream-edmm_stepType-avgua.nc")

# Print variable names
print(ds4.variables)

Frozen({'u10': <xarray.Variable (number: 10, valid_time: 576, latitude: 361, longitude: 720)> Size: 6GB
[1497139200 values with dtype=float32]
Attributes: (12/32)
    GRIB_paramId:                             165
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      259920
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgua
    ...                                       ...
    GRIB_totalNumber:                         10
    GRIB_units:                               m s**-1
    long_name:                                10 metre U wind component
    units:                                    m s**-1
    standard_name:                            unknown
    GRIB_surface:                             0.0, 'v10': <xarray.Variable (number: 10, valid_time: 576, latitude: 361, longitude: 720)> Size: 6GB
[1497139200 values with dtype=float32]
Attributes: 

### Gif Function

In [4]:
def create_wind_gif(
    netcdf_path,
    output_file="era5_wind_flow.gif",
    time_range=None,
    num_particles=5000,
    trail_length=25,
    drop_rate=0.08,
    speed_factor=0.6,
    frames_per_timestep=10,
    figsize=(16, 8),
    fps=30,
    dpi=100,
    show_continents=True,
    wrap_longitude=True
):
    """
    Create an animated GIF of wind flow from ERA5 data.
    
    Parameters:
    -----------
    netcdf_path : str
        Path to the ERA5 NetCDF file
    output_file : str
        Output GIF filename (default: 'era5_wind_flow.gif')
    time_range : tuple, int, or None
        Specify the time range to animate:
        - Tuple of (start, end): ('2010-01-01', '2010-01-31') - inclusive date range
        - Single int: Use specific number of timestamps starting from beginning
        - None: Uses first 12 timestamps
    num_particles : int
        Number of particles (default: 5000, lower for smoother animation)
    trail_length : int
        Length of particle trails in frames (default: 25)
    drop_rate : float
        Particle drop rate 0-1 (default: 0.08)
    speed_factor : float
        Wind speed multiplier (default: 0.6)
    frames_per_timestep : int
        Number of animation frames per timestamp (default: 10)
    figsize : tuple
        Figure size in inches (default: (16, 8))
    fps : int
        Frames per second (default: 30)
    dpi : int
        Output resolution (default: 100)
    show_continents : bool
        Show continent outlines (default: True)
    wrap_longitude : bool
        Allow particles to wrap around at longitude boundaries (default: True)
        Set to False to prevent particles from crossing left-right edges
    
    Returns:
    --------
    dict : Statistics about the GIF
    
    """
    
    print("🌬️  ERA5 Wind Flow - GIF Generator")
    print("=" * 60)
    
    # Load ERA5 data
    print("\n📂 Loading NetCDF data...")
    ds = xr.open_dataset(netcdf_path)
    
    # Extract variables - handle ensemble dimension if present
    u = ds["u10"]
    v = ds["v10"]
    
    # Check for ensemble dimension and take mean if it exists
    if "number" in u.dims:
        print("✓ Ensemble data detected, computing ensemble mean...")
        u = u.mean(dim="number")
        v = v.mean(dim="number")
    else:
        print("✓ Single realization data (no ensemble dimension)")
    
    time = ds["valid_time"].values
    time_orig = time.copy()  # Keep original for error messages
    lat = ds["latitude"].values
    lon = ds["longitude"].values
    
    print(f"✓ Original shape: {u.shape}")
    print(f"✓ Full time range: {str(time[0])[:10]} to {str(time[-1])[:10]}")
    print(f"✓ Total timestamps in file: {len(time)}")
    
    # Show sample of timestamps to understand the data frequency
    if len(time) >= 10:
        print(f"✓ Sample timestamps: {[str(t)[:19] for t in time[:10]]}")
    
    # Detect data frequency
    if len(time) > 1:
        time_diff = (time[1] - time[0]) / np.timedelta64(1, 'h')
        print(f"✓ Detected frequency: ~{time_diff:.1f} hours between timestamps")

    start_date, end_date = time_range
    
    # Convert timestamps to dates (strip time) for filtering
    time_dates = time.astype('datetime64[D]')
    start_date_only = np.datetime64(start_date, 'D')
    end_date_only = np.datetime64(end_date, 'D')
    
    # Filter: include all timestamps whose date falls within the range (inclusive)
    time_mask = (time_dates >= start_date_only) & (time_dates <= end_date_only)
    
    u = u[time_mask]
    v = v[time_mask]
    time = time[time_mask]
    
    print(f"✓ Filtered: Range {start_date} to {end_date} ({len(time)} timestamps)")
    
    # Show what timestamps were actually captured
    if len(time) > 0 and len(time) <= 20:
        print(f"✓ Captured timestamps: {[str(t)[:19] for t in time]}")
    elif len(time) > 20:
        print(f"✓ First 5 timestamps: {[str(t)[:19] for t in time[:5]]}")
        print(f"✓ Last 5 timestamps: {[str(t)[:19] for t in time[-5:]]}")
    
    num_timesteps = len(time)
    
    # Check if we have valid data
    if num_timesteps == 0:
        print(f"\n❌ ERROR: No timestamps found for the specified filter!")
        print(f"   Available time range: {str(time_orig[0])[:10]} to {str(time_orig[-1])[:10]}")
        print(f"   First few timestamps: {[str(t)[:19] for t in time_orig[:5]]}")
        raise ValueError("No timestamps match the specified time filter. Check your time_range parameter.")
    
    print(f"✓ Animation will use {num_timesteps} timestamps")
    
    # Convert longitudes from 0–360 to -180–180
    lon = ((lon + 180) % 360) - 180
    lon_order = np.argsort(lon)
    lon = lon[lon_order]
    u = u[:, :, lon_order]
    v = v[:, :, lon_order]
    
    # Latitude is descending, reverse to ascending
    if lat[0] > lat[-1]:
        lat = lat[::-1]
        u = u[:, ::-1, :]
        v = v[:, ::-1, :]
    
    print(f"✓ Grid: {u.shape[1]} x {u.shape[2]} (lat x lon)")
    
    # Colormap
    colors = ['#1e3a8a', '#3b82f6', '#22d3ee', '#10b981', '#fbbf24', '#f97316', '#dc2626']
    cmap = LinearSegmentedColormap.from_list('vangogh', colors, N=100)
    
    def get_wind_at_location(lons, lats, time_idx):
        """Get wind at particle locations"""
        lat_norm = (lats - lat.min()) / (lat.max() - lat.min())
        lon_norm = (lons - lon.min()) / (lon.max() - lon.min())
        
        lat_indices = np.clip((lat_norm * (len(lat) - 1)).astype(int), 0, len(lat) - 1)
        lon_indices = np.clip((lon_norm * (len(lon) - 1)).astype(int), 0, len(lon) - 1)
        
        u_vals = u[time_idx].values[lat_indices, lon_indices]
        v_vals = v[time_idx].values[lat_indices, lon_indices]
        
        u_vals = np.nan_to_num(u_vals, nan=0.0)
        v_vals = np.nan_to_num(v_vals, nan=0.0)
        
        return u_vals, v_vals
    
    print(f"\n🎬 Setting up animation with {num_particles:,} particles...")
    
    # Setup figure with cartopy if continents requested
    if show_continents:
        fig = plt.figure(figsize=figsize, facecolor='#0a1929')
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
        ax.set_facecolor('#0a1929')
        ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
        
        # Add continent outlines
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='white', alpha=0.3)
        ax.add_feature(cfeature.BORDERS, linewidth=0.3, edgecolor='white', alpha=0.2)
        ax.add_feature(cfeature.LAND, facecolor='none', edgecolor='white', alpha=0.1)
        
        ax.axis('off')
    else:
        fig, ax = plt.subplots(figsize=figsize, facecolor='#0a1929')
        ax.set_facecolor('#0a1929')
        ax.set_xlim(-180, 180)
        ax.set_ylim(-90, 90)
        ax.set_aspect('equal')
        ax.axis('off')
    
    # Initialize particles
    particles = {
        'lon': np.random.uniform(-180, 180, num_particles),
        'lat': np.random.uniform(-90, 90, num_particles),
        'trail_lon': [[] for _ in range(num_particles)],
        'trail_lat': [[] for _ in range(num_particles)]
    }
    
    scatter = ax.scatter([], [], s=2.5, c=[], cmap=cmap, vmin=0, vmax=20, alpha=0.7)
    lines = [ax.plot([], [], linewidth=1.2, alpha=0.3)[0] for _ in range(num_particles)]
    title_text = ax.text(0.5, 1.05, '', transform=ax.transAxes, 
                         color='white', fontsize=16, weight='bold', ha='center')
    
    def update(frame):
        # Determine which timestamp to use
        time_idx = min(frame // frames_per_timestep, num_timesteps - 1)
        
        # Get wind at particle locations
        u_wind, v_wind = get_wind_at_location(particles['lon'], particles['lat'], time_idx)
        speed = np.sqrt(u_wind**2 + v_wind**2)
        
        # Move particles
        particles['lon'] += u_wind * speed_factor
        particles['lat'] += v_wind * speed_factor * 0.5
        
        # Handle longitude wrapping
        if wrap_longitude:
            # Wrap longitude (allows crossing at edges)
            particles['lon'] = ((particles['lon'] + 180) % 360) - 180
        else:
            # Clip longitude (prevents crossing, resets particles at edges)
            edge_mask = (particles['lon'] < -180) | (particles['lon'] > 180)
            particles['lon'] = np.clip(particles['lon'], -180, 180)
            # Reset particles that hit edges
            particles['lon'][edge_mask] = np.random.uniform(-180, 180, edge_mask.sum())
            particles['lat'][edge_mask] = np.random.uniform(-90, 90, edge_mask.sum())
            for i in np.where(edge_mask)[0]:
                particles['trail_lon'][i] = []
                particles['trail_lat'][i] = []
        
        # Clip latitude
        particles['lat'] = np.clip(particles['lat'], -90, 90)
        
        # Update trails
        for i in range(num_particles):
            particles['trail_lon'][i].append(particles['lon'][i])
            particles['trail_lat'][i].append(particles['lat'][i])
            if len(particles['trail_lon'][i]) > trail_length:
                particles['trail_lon'][i].pop(0)
                particles['trail_lat'][i].pop(0)
        
        # Drop and reset particles
        reset_mask = np.random.random(num_particles) < drop_rate
        particles['lon'][reset_mask] = np.random.uniform(-180, 180, reset_mask.sum())
        particles['lat'][reset_mask] = np.random.uniform(-90, 90, reset_mask.sum())
        
        for i in np.where(reset_mask)[0]:
            particles['trail_lon'][i] = []
            particles['trail_lat'][i] = []
        
        # Update visualization
        scatter.set_offsets(np.c_[particles['lon'], particles['lat']])
        scatter.set_array(speed)
        
        # Update trail lines
        for i, line in enumerate(lines):
            if len(particles['trail_lon'][i]) > 1:
                line.set_data(particles['trail_lon'][i], particles['trail_lat'][i])
                color = cmap(min(speed[i] / 20, 1.0))
                line.set_color(color)
                line.set_alpha(0.35)
            else:
                line.set_data([], [])
        
        # Update title
        title_text.set_text(f'ERA5 Wind Flow - {str(time[time_idx])[:16]}')
        
        return [scatter, title_text] + lines
    
    # Calculate total frames
    total_frames = num_timesteps * frames_per_timestep
    print(f"✓ Total frames: {total_frames} ({num_timesteps} timesteps × {frames_per_timestep} frames/timestep)")
    print(f"✓ Duration: {total_frames / fps:.1f} seconds at {fps} FPS")
    
    print("\n⏱️  Creating animation (this may take 1-3 minutes)...")
    ani = animation.FuncAnimation(fig, update, frames=total_frames, 
                                  interval=1000/fps, blit=True)
    
    # Save as GIF
    print(f"💾 Saving GIF to {output_file}...")
    ani.save(output_file, writer='pillow', fps=fps, dpi=dpi)
    plt.close(fig)
    
    # Calculate file size
    import os
    file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
    
    # Statistics
    stats = {
        'output_file': output_file,
        'file_size_mb': f"{file_size_mb:.2f} MB",
        'particles': num_particles,
        'timesteps': num_timesteps,
        'total_frames': total_frames,
        'duration_seconds': f"{total_frames / fps:.1f}",
        'fps': fps,
        'trail_length': trail_length,
        'drop_rate': drop_rate,
        'time_range': f"{str(time[0])[:16]} to {str(time[-1])[:16]}"
    }
    
    print(f"\n✅ GIF saved: {output_file}")
    print("\n📊 GIF Stats:")
    for key, value in stats.items():
        if key != 'output_file':
            print(f"  • {key.replace('_', ' ').title()}: {value}")
    
    return stats

### Gif Creations

In [8]:
# Monthly overview using Monthly Averaged Reanalysis
if __name__ == "__main__":
    print("\n" + "="*60)
    print("Monthly Averaged Reanalysis - January to December 2010")
    print("="*60)
    create_wind_gif(
        "data_stream-moda_stepType-avgua.nc",
        output_file="wind_flow_2010.gif",
        time_range=('2010-01-01', '2010-12-31'),
        speed_factor=0.5,
        frames_per_timestep=30,
        fps=30,
        show_continents=True,
        wrap_longitude=False
    )


Monthly Averaged Reanalysis - January to December 2010
🌬️  ERA5 Wind Flow - GIF Generator

📂 Loading NetCDF data...
✓ Single realization data (no ensemble dimension)
✓ Original shape: (72, 721, 1440)
✓ Full time range: 2010-01-01 to 2015-12-01
✓ Total timestamps in file: 72
✓ Sample timestamps: ['2010-01-01T00:00:00', '2010-02-01T00:00:00', '2010-03-01T00:00:00', '2010-04-01T00:00:00', '2010-05-01T00:00:00', '2010-06-01T00:00:00', '2010-07-01T00:00:00', '2010-08-01T00:00:00', '2010-09-01T00:00:00', '2010-10-01T00:00:00']
✓ Detected frequency: ~744.0 hours between timestamps
✓ Filtered: Range 2010-01-01 to 2010-12-31 (12 timestamps)
✓ Captured timestamps: ['2010-01-01T00:00:00', '2010-02-01T00:00:00', '2010-03-01T00:00:00', '2010-04-01T00:00:00', '2010-05-01T00:00:00', '2010-06-01T00:00:00', '2010-07-01T00:00:00', '2010-08-01T00:00:00', '2010-09-01T00:00:00', '2010-10-01T00:00:00', '2010-11-01T00:00:00', '2010-12-01T00:00:00']
✓ Animation will use 12 timestamps
✓ Grid: 721 x 1440 (lat 

In [10]:
# January 2010 overview using Monthly Averaged Reanalysis by Hour of Day
if __name__ == "__main__":
    print("\n" + "="*60)
    print("Monthly Averaged Reanalysis by Hour of Day - January 2010")
    print("="*60)
    create_wind_gif(
        "data_stream-edmm_stepType-avgua.nc",
        output_file="wind_flow_january_2010.gif",
        time_range=('2010-01-01', '2010-01-31'),
        speed_factor=0.5,
        frames_per_timestep=30,
        fps=30,
        show_continents=True,
        wrap_longitude=False
    )


Monthly Averaged Reanalysis by Hour of Day - January 2010
🌬️  ERA5 Wind Flow - GIF Generator

📂 Loading NetCDF data...
✓ Ensemble data detected, computing ensemble mean...
✓ Original shape: (576, 361, 720)
✓ Full time range: 2010-01-01 to 2015-12-01
✓ Total timestamps in file: 576
✓ Sample timestamps: ['2010-01-01T00:00:00', '2010-01-01T03:00:00', '2010-01-01T06:00:00', '2010-01-01T09:00:00', '2010-01-01T12:00:00', '2010-01-01T15:00:00', '2010-01-01T18:00:00', '2010-01-01T21:00:00', '2010-02-01T00:00:00', '2010-02-01T03:00:00']
✓ Detected frequency: ~3.0 hours between timestamps
✓ Filtered: Range 2010-01-01 to 2010-01-31 (8 timestamps)
✓ Captured timestamps: ['2010-01-01T00:00:00', '2010-01-01T03:00:00', '2010-01-01T06:00:00', '2010-01-01T09:00:00', '2010-01-01T12:00:00', '2010-01-01T15:00:00', '2010-01-01T18:00:00', '2010-01-01T21:00:00']
✓ Animation will use 8 timestamps
✓ Grid: 361 x 720 (lat x lon)

🎬 Setting up animation with 5,000 particles...
✓ Total frames: 240 (8 timesteps × 

In [11]:
# Monthly overview using Monthly Averaged Ensemble Members by Hour of Day
if __name__ == "__main__":
    print("\n" + "="*60)
    print("Monthly Averaged Ensemble Members by Hour of Day - January to December 2010")
    print("="*60)
    create_wind_gif(
        "data_stream-edmm_stepType-avgua.nc",
        output_file="wind_flow_2010_ensemble.gif",
        time_range=('2010-01-01', '2010-12-31'),
        speed_factor=0.5,
        frames_per_timestep=30,
        fps=30,
        show_continents=True,
        wrap_longitude=False
    )


Monthly Averaged Ensemble Members by Hour of Day - January to December 2010
🌬️  ERA5 Wind Flow - GIF Generator

📂 Loading NetCDF data...
✓ Ensemble data detected, computing ensemble mean...
✓ Original shape: (576, 361, 720)
✓ Full time range: 2010-01-01 to 2015-12-01
✓ Total timestamps in file: 576
✓ Sample timestamps: ['2010-01-01T00:00:00', '2010-01-01T03:00:00', '2010-01-01T06:00:00', '2010-01-01T09:00:00', '2010-01-01T12:00:00', '2010-01-01T15:00:00', '2010-01-01T18:00:00', '2010-01-01T21:00:00', '2010-02-01T00:00:00', '2010-02-01T03:00:00']
✓ Detected frequency: ~3.0 hours between timestamps
✓ Filtered: Range 2010-01-01 to 2010-12-31 (96 timestamps)
✓ First 5 timestamps: ['2010-01-01T00:00:00', '2010-01-01T03:00:00', '2010-01-01T06:00:00', '2010-01-01T09:00:00', '2010-01-01T12:00:00']
✓ Last 5 timestamps: ['2010-12-01T09:00:00', '2010-12-01T12:00:00', '2010-12-01T15:00:00', '2010-12-01T18:00:00', '2010-12-01T21:00:00']
✓ Animation will use 96 timestamps
✓ Grid: 361 x 720 (lat x l

In [12]:
# January 2010 overview using Monthly Averaged Ensemble Members by Hour of Day
if __name__ == "__main__":
    print("\n" + "="*60)
    print("Monthly Averaged Ensemble Members by Hour of Day - January to December 2010")
    print("="*60)
    create_wind_gif(
        "data_stream-edmm_stepType-avgua.nc",
        output_file="wind_flow_january_2010_ensemble.gif",
        time_range=('2010-01-01', '2010-01-31'),
        speed_factor=0.5,
        frames_per_timestep=30,
        fps=30,
        show_continents=True,
        wrap_longitude=False
    )


Monthly Averaged Ensemble Members by Hour of Day - January to December 2010
🌬️  ERA5 Wind Flow - GIF Generator

📂 Loading NetCDF data...
✓ Ensemble data detected, computing ensemble mean...
✓ Original shape: (576, 361, 720)
✓ Full time range: 2010-01-01 to 2015-12-01
✓ Total timestamps in file: 576
✓ Sample timestamps: ['2010-01-01T00:00:00', '2010-01-01T03:00:00', '2010-01-01T06:00:00', '2010-01-01T09:00:00', '2010-01-01T12:00:00', '2010-01-01T15:00:00', '2010-01-01T18:00:00', '2010-01-01T21:00:00', '2010-02-01T00:00:00', '2010-02-01T03:00:00']
✓ Detected frequency: ~3.0 hours between timestamps
✓ Filtered: Range 2010-01-01 to 2010-01-31 (8 timestamps)
✓ Captured timestamps: ['2010-01-01T00:00:00', '2010-01-01T03:00:00', '2010-01-01T06:00:00', '2010-01-01T09:00:00', '2010-01-01T12:00:00', '2010-01-01T15:00:00', '2010-01-01T18:00:00', '2010-01-01T21:00:00']
✓ Animation will use 8 timestamps
✓ Grid: 361 x 720 (lat x lon)

🎬 Setting up animation with 5,000 particles...
✓ Total frames: 2